In [ ]:
#@title Copyright 2022 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Cloud  Backed Earth Engine Tables

In [1]:
import pandas as pd
from google.cloud import storage
import os
import json

## Start an authorized session

To be able to make an Earth Engine asset in your user folder, you need to be able to authenticate as yourself when you make the request.  You can use credentials from the Earth Engine authenticator to start an [`AuthorizedSession`](https://google-auth.readthedocs.io/en/master/reference/google.auth.transport.requests.html#google.auth.transport.requests.AuthorizedSession).  You can then use the `AuthorizedSession` to send requests to Earth Engine.

In [5]:
import ee
from google.auth.transport.requests import AuthorizedSession

#ee.Authenticate()  #  or 
!earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

Fetching credentials using gcloud
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=XP5sIirtvsC3o2JrYSyzjEZkXDlD8t&access_type=offline&code_challenge=KowuvQeZJ9gotoxTesZxh82ofXfB9FhEl2Z-zXlUV54&code_challenge_method=S256


Credentials saved to file: [/Users/christiannilsen/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Successfully saved authorization token.


## Get image information from gcp
Get list of images and their respective uris from the gcp data bucket


In [6]:
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    # Note: The call returns a response only when the iterator is consumed.
    blob_list = []
    for blob in blobs:
        blob_list.append(blob.name)

    if delimiter:
        print("Prefixes:")
        for prefix in blobs.prefixes:
            blob_list.append([prefix])
    
    return blob_list


In [53]:
table_name = 'nhdplus' 

In [54]:
BUCKET_NAME = 'swhm-bq-exports'
blobsout = list_blobs_with_prefix(BUCKET_NAME,'cocs/'+table_name+'/'+table_name)

In [55]:
df = pd.DataFrame(blobsout, columns=['blob_name'])
df['uri'] = 'gs://'+BUCKET_NAME+'/'+df['blob_name']
(df)

,blob_name,uri
0,cocs/nhdplus/nhdplus000000000000.csv,gs://swhm-bq-exports/cocs/nhdplus/nhdplus00000...


In [56]:

file_names = [os.path.splitext(os.path.basename(file_path))[0] for file_path in df['blob_name']]
df["asset_name"] = table_name

df

,blob_name,uri,asset_name
0,cocs/nhdplus/nhdplus000000000000.csv,gs://swhm-bq-exports/cocs/nhdplus/nhdplus00000...,nhdplus


In [57]:
asset_name = df['asset_name'][0]
uri = df['uri'][0]
cmd = 'earthengine upload table --asset_id=projects/ee-swhm/assets/production_feature_collections/{} {}'.format(asset_name,uri) 
!{cmd}

Started upload task with ID: GYQAXWNJZ2LPGKJI4HGSWUO5
